In [9]:

import pandas as pd 
import json 

# data_path_tamplate = 'Result/Mixtral_template_RULE_{key}_Mixtral_8bit.jsonl'
# data_path_tamplate = 'Result/Alpaca_template_RULE_{key}_Llama-70B-8bit.jsonl'
# data_path_tamplate = 'Result/gpt_results/result_{key}_gpt-4-1106-preview.csv'
data_path_mainq = 'Result/Mixtral_template_cot1_(ThreeParagraphReasoning)_RULE_mainq_Mixtral_half.jsonl'
data_path_subq = 'Result/Mixtral_template_cot1_(ThreeParagraphReasoning)_RULE_subq_all_Mixtral_half.jsonl'
path_key_list = ['mainq', 'subq_all']

subq_all = pd.read_json('../Data/RULE/RULE_subq_all.jsonl', lines=True)

only_mainq = False

def extract_first_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()]))-1

def extract_last_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()])[-1])-1
    
def correctness_matching(subq_all: pd.DataFrame, subq_id: str):
    if subq_id in subq_all['id_string'].values:
        return subq_all[subq_all['id_string']==subq_id]['main_option_correctness'].values[0]
    else: 
        return None

def extract_mainq_id_string(subq_id: str):
    return subq_id.split('_sub')[0]

def extract_answer_from_cot_response(response: str):
    low_response = response.lower()
    # 'answer: '를 찾는다
    answer_start_idx = low_response.rfind('answer:')
    # 특수문자 제거: [, ]
    low_response = low_response.replace('[', '').replace(']', '')
    # 'answer: '가 없으면 -1을 리턴
    
    extraction_type = ''
    
    if answer_start_idx == -1:
        extraction_type = 'no "answer:"'
        return -1#, extraction_type
    # 'answer: '뒤에서부터 for문을 돌린다
    try: 
        after_answer = low_response[answer_start_idx+len('answer:'):].strip()
        # print(f"after_answer: {after_answer}")
        if after_answer[0].isdigit():
            extraction_type = 'Answer format digit'
            return int(after_answer[0])-1#, extraction_type
        elif after_answer[0] in ['a', 'b', 'c', 'd']:
            extraction_type = 'Answer format alphabet'
            return ord(after_answer[0])-ord('a')#, extraction_type
        elif after_answer[0] == 'n' and after_answer[1].isdigit():
            extraction_type = 'Answer format: "n1"'
            return int(after_answer[1])-1#, extraction_type
        # 'answer: '뒤에 숫자가 한개라도 있는 경우 
        elif any(char.isdigit() for char in after_answer):
            for i in range(len(after_answer)):
                if after_answer[i].isdigit():
                    extraction_type = 'error (maybe nothing after "answer:")'
                    return int(after_answer[i])-1#, extraction_type
        else:
            return -1
    except:
        print(f"after_answer: {after_answer}")
        extraction_type = 'error (maybe nothing after "answer:")'
        return -1#, extraction_type
        
        
    
mainq_result_path = data_path_mainq
if 'jsonl' in mainq_result_path:
    mainq_result = pd.read_json(mainq_result_path, lines=True)
elif 'csv' in mainq_result_path:
    mainq_result = pd.read_csv(mainq_result_path)

mainq_result['prediction'] = mainq_result['inference'].apply(extract_answer_from_cot_response)#(extract_last_num_in_string)
correct = 0
for i in range(len(mainq_result)):
    if mainq_result.label[i] == mainq_result.prediction[i]:
        correct += 1
print(f'mainq_result, accuracy: {correct/len(mainq_result)}')

if only_mainq == False:
    subq_result_path = data_path_subq
    if 'jsonl' in subq_result_path:
        subq_result = pd.read_json(subq_result_path, lines=True)
    elif 'csv' in subq_result_path:
        subq_result = pd.read_csv(subq_result_path)
    
    if 'jsonl' in subq_result_path:
        # data['main_option_correctness'] 열 만들기
        subq_result['main_option_correctness'] = subq_result['id_string'].apply(lambda x: correctness_matching(subq_all, x))
        subq_result['prediction'] = subq_result['inference'].apply(extract_answer_from_cot_response)
        subq_result['mainq_id'] = subq_result['id_string'].apply(extract_mainq_id_string)
        correct = 0
        for i in range(len(subq_result)):
            if subq_result.label[i] == subq_result.prediction[i]:
                correct += 1
        print(f'subq_result, accuracy: {correct/len(subq_result)}')

    elif 'csv' in mainq_result_path:
        print(f'mainq_result, accuracy: {len(mainq_result[mainq_result.is_correct == True])/len(mainq_result)}')
        print(f'subq_result, accuracy: {len(subq_result[subq_result.is_correct == True])/len(subq_result)}')
        
        correct = 0
        for i in range(len(mainq_result)):
            # mainq_result가 맞았을 때
            if mainq_result.is_correct[i]:
                # mainq_result.id_string[i]와 같은 subq_result.mainq_id 가진 행을 찾아서 모두 정답을 확인
                subq_result_sub = subq_result[subq_result['main_id']==mainq_result.id[i]]
                
                if True in subq_result_sub.is_correct.values:
                    if subq_result_sub.is_correct.value_counts()[True] == len(subq_result_sub):
                        correct += 1
                    
        print(f'Consistency: {correct/len(mainq_result)}')

after_answer: 
after_answer: 
after_answer: 
after_answer: 
after_answer: 
after_answer: 
mainq_result, accuracy: 0.5514316012725344


ValueError: invalid literal for int() with base 10: '₅'

In [8]:
mainq_result

,context,question,answers,label,id_string,input_string,inference,main_option_correctness,prediction
0,Clark: Our local community theater often produ...,The reasoning in Clark's argument is most vuln...,[uses as evidence a source that there is reaso...,2,train_1348,[INST] Below is an instruction that describes ...,"Let's think step by step. \nFirst, Clark state...",NaN,2
1,Electronic cigarettes should not be subject to...,Which one of the following most weakens the ar...,[The current regulations are designed to preve...,0,train_439,[INST] Below is an instruction that describes ...,Let's think step by step. \nThe argument is th...,NaN,1
2,Politician: Most of those at the meeting were ...,Which one of the following describes a flaw in...,[It rejects an argument merely on the grounds ...,0,train_1030,[INST] Below is an instruction that describes ...,"Let's think step by step. \nFirst, the politic...",NaN,0
3,Journalists agree universally that lying is ab...,Which one of the following is most supported b...,[Since lying is permissible in some situations...,1,train_314,[INST] Below is an instruction that describes ...,"Let's think step by step. \nFirst, the passage...",NaN,1
4,Teachers should not do anything to cause their...,The conclusion is properly drawn if which one ...,"[Students respect honesty above all else., Stu...",1,train_79,[INST] Below is an instruction that describes ...,Let's think step by step. \nThe passage sugges...,NaN,0
...,...,...,...,...,...,...,...,...,...
938,Criticism that the press ignores positive stor...,It can properly be inferred from the passage t...,"[comes from its ability to tell a good story, ...",1,train_973,[INST] Below is an instruction that describes ...,"Let's think step by step. \nFirst, the passage...",NaN,-1
939,Traveler: Southern Airways has a far worse saf...,"Which of the following, if true, casts the mos...",[Although the frequency of accidents is greate...,1,train_15,[INST] Below is an instruction that describes ...,Let's think step by step. \nThe traveler'S arg...,NaN,3
940,A certain cultivated herb is one of a group of...,"In evaluating the argument, it would be most i...",[Whether the herb can thrive in soil that does...,3,train_26,[INST] Below is an instruction that describes ...,Let's think step by step. \nThe argument is ce...,NaN,3
941,Commentator: The quality of health care is dec...,"Which one of the following, if true, most weak...",[Most patients visiting a physician are suffer...,3,train_1231,[INST] Below is an instruction that describes ...,Let's think step by step. \nThe commentator' s...,NaN,-1


In [159]:
def extract_answer_from_cot_response(response: str):
    low_response = response.lower()
    # 'answer: '를 찾는다
    answer_start_idx = low_response.rfind('answer:')
    # 'answer: '가 없으면 -1을 리턴
    if answer_start_idx == -1:
        return -1
    # 'answer: '뒤에서부터 for문을 돌린다
    after_answer = low_response[answer_start_idx+len('answer:'):].strip()
    # print(f"after_answer: {after_answer}")
    if after_answer[0].isdigit():
        return int(after_answer[0])-1
    elif after_answer[0] in ['a', 'b', 'c', 'd']:
        return ord(after_answer[0])-ord('a')
    elif after_answer[0] == 'n' and after_answer[1].isdigit():
        return int(after_answer[1])-1
    # 'answer: '뒤에 숫자가 한개라도 있는 경우 
    elif any(char.isdigit() for char in after_answer):
        for i in range(len(after_answer)):
            if after_answer[i].isdigit():
                return int(after_answer[i])-1
    else:
        return -1

'''
 - Answer: 숫자         # 23
 - 답이 없음            # 4
 - Answer: 알파벳       # 9    (Answer: 이후의 첫 알파벳에 매칭하면 되는 경우)
 - Answer: n숫자        # 2
 - Answer: 문자열~숫자  # 1     (Answer: 이후의 첫 숫자에 매칭하면 되는 경우)
 - 답이 둘 이상인 경우  # 2
 
 -> Policy: 
    - inference에서 가장 마지막으로 등장하는 'Answer:'를 찾고, 
        1. 그 뒤에 공백을 제외하고 첫 글자가 숫자인 경우, <prediction = 그 숫자-1>
        2. 그 뒤에 공백을 제외하고 첫 글자가 A,B,C,D 중 하나의 알파벳인 경우, <prediction = 알파벳에 해당하는 숫자 (A=1, B=2)-1>
        3. 그 뒤에 공백을 제외하고 첫 글자가 n숫자인 경우, <prediction = 그 숫자-1>
        4. 위에 해당되지 않으면 -1을 리턴
    
'''
num_index = 8
print(f'{mainq_result.inference.values[num_index]}')
print(f'Extracted: {extract_answer_from_cot_response(mainq_result.inference.values[num_index])}')


Let's think step by step. 
The researchers' argument is based on the assumption that if radiation from nuclear plants had any negative health effects, there would be a noticeable difference in death rates between areas near the nuclear plants and areas without them. However, they found no such difference, and thus concluded that there are no health hazardous effects. 
Now, let's examine the given options to see which one, if true，most seriously weakens this argument:

(1) Only a smallnumber of areas havenuclear weapons plants. This option suggests that the sample size of areas with nuclear weapons plants might be too small to draw a definitive conclusion. However, it doesn't necessarily weaken the argument because even if the sample size is small, if there's no difference in death rate, it still suggests no significant health hazard.

(2) Nuclear powerm plants were notincluded in the study. This is a more relevant option. The study only considered nuclear weapons plants, not nuclear po

In [90]:
# data에서 main_option_correctness 값이 True인 것만 추출하기 
data_true = subq_result[subq_result['main_option_correctness']==True]
correct = 0
for i in range(len(data_true)):
    if data_true.label.values[i] == data_true.prediction.values[i]:
        correct += 1
print(f'data_true, accuracy: {correct/len(data_true)}')

# data에서 main_option_correctness 값이 False인 것만 추출하기
data_false = subq_result[subq_result['main_option_correctness']==False]
correct = 0
for i in range(len(data_false)):
    if data_false.label.values[i] == data_false.prediction.values[i]:
        correct += 1
print(f'data_false, accuracy: {correct/len(data_false)}')

correct = 0
for i in range(len(mainq_result)):
    # mainq_result가 맞았을 때
    if mainq_result.label[i] == mainq_result.prediction[i]:
        # mainq_result.id_string[i]와 같은 subq_result.mainq_id 가진 행을 찾아서 모두 정답을 확인
        subq_result_sub = subq_result[subq_result['mainq_id']==mainq_result.id_string[i]]
        
        correct_ = True
        for j in range(len(subq_result_sub)):
            if subq_result_sub.label.values[j] != subq_result_sub.prediction.values[j]:
                correct_ = False
                break
        if correct_:
            correct += 1

print(f'Consistency: {correct/len(mainq_result)}')

data_true, accuracy: 0.8535031847133758
data_false, accuracy: 0.609107303877367
Consistency: 0.19300106044538706


In [2]:
# 아래의 각 값에 가짜 값을 부여함 
inferenced = []
context = 'context'
question = 'question'
answers = ['answer1', 'answer2', 'answer3', 'answer4']
label = 0
id_string = 'id_string'
input_string = 'input_string'
inference = 'inference'
main_option_correctness = True

for i in range(3):
    inferenced.append({
                "context": context,
                "question": question,
                "answers": answers,
                "label": label,
                "id_string": id_string,
                "input_string": input_string,
                "inference": inference,
                "main_option_correctness": main_option_correctness,
            })


In [ ]:
import os 

output_filepath = os.path.join(output_path, f'{output_key}.jsonl')
if output_key == "":
    output_key = data_path.replace(".jsonl", "_inference.jsonl")
with open(output_filepath, "w") as f:
    for x in inferenced:
        f.write(json.dumps(x) + "\n")

In [23]:
dpath = '/hdd/hjl8708/workspace/Data/RULE/RULE_subq_all.jsonl'

import pandas as pd

subq_data = pd.read_json(dpath, lines=True)

# subq_data에서 question을 sampling (10개)
import random
subq_data_correct = subq_data[subq_data['main_option_correctness']==True]
subq_data_incorrect = subq_data[subq_data['main_option_correctness']==False]

questions = list((subq_data_correct.question.values))
random.seed(42)
random.shuffle(questions)
questions = questions[:8]

print("<selective>\n"+\
    "\n".join([f"{ix}: {x}" for ix, x in enumerate(questions)]))


questions = list((subq_data_incorrect.question.values))
random.seed(42)
random.shuffle(questions)
questions = questions[:8]

print("<eliminative>\n"+\
    "\n".join([f"{ix}: {x}" for ix, x in enumerate(questions)]))

<selective>
0: How does the fact that identical twins who grow up together tend to develop different beliefs, tastes, and careers in order to differentiate themselves from each other weaken the psychologist's argument?
1: How does the economist's argument use the technique of appealing to an analogy in order to indicate the futility of a course of action?
2: Why is it valid to conclude that the environmentalist's argument depends on the assumption that the reactions of food producers in other countries to laws requiring labeling of foods containing genetically altered ingredients are likely to be similar to the reactions of food producers in the United Kingdom?
3: How does the fact that the scientists' investigation would conclusively show whether or not the mountain has any geological faults strengthen the geologist's argumentation?
4: What evidence does the argument cite to support the claim that there is a lack of evidence supporting a particular explanation and further evidence tha

In [21]:
questions_zipped



In [8]:
import json 
import os 
result_files = [f'Result/Mixtral_template_RULE_mainq_Mixtral_4bit_{x}.jsonl' for x in range(0, 3)]


all_inferenced = []
for rank in range(3):
    rank_output_filepath = result_files[rank]
    with open(rank_output_filepath, "r") as f:
        for line in f:
            all_inferenced.append(json.loads(line))

# 최종 파일 저장
final_output_filepath = os.path.join('Result/Mixtral_template_RULE_mainq_Mixtral_4bit_final.jsonl')
with open(final_output_filepath, "w") as f:
    for x in all_inferenced:
        f.write(json.dumps(x) + "\n")

In [31]:
subq_all = pd.read_json('../Data/RULE/RULE_subq_all.jsonl', lines=True)
subq_all

,main_id,context,question,label,main_question,main_option_correctness,id_string,answers
0,train_1,Patient: Pharmacists maintain that doctors sho...,What is the rationale for the patient's argume...,2,{'context': 'Patient: Pharmacists maintain tha...,True,train_1_sub1,[Lack of pharmacists' knowledge was not highli...
1,train_1,Patient: Pharmacists maintain that doctors sho...,What is the evidence that the patient's argume...,0,{'context': 'Patient: Pharmacists maintain tha...,False,train_1_sub4,[Lack of pharmacists' knowledge was not highli...
2,train_11,Biologist: A careful study of the behavior of ...,Why is the biologist's argument considered vul...,0,{'context': 'Biologist: A careful study of the...,True,train_11_sub1,[The writer refers here to focusing on who wro...
3,train_11,Biologist: A careful study of the behavior of ...,Why is the reasoning in the biologist's argume...,0,{'context': 'Biologist: A careful study of the...,False,train_11_sub2,"[Here, the author points towards conclusions o..."
4,train_10,Detective: Laser-printer drums are easily dama...,Explain why the fact that most suspicious docu...,1,{'context': 'Detective: Laser-printer drums ar...,False,train_10_sub1,[This weakens the argument because it tells th...
...,...,...,...,...,...,...,...,...
2998,train_1085,Human rights activists have urged consumers to...,Explain why the principle that it is more impo...,3,{'context': 'Human rights activists have urged...,False,train_1085_sub3,[There is no contradiction because the whole t...
2999,train_1263,PhishCo runs a number of farms in the arid pro...,Why does the fact that the government of Nufa ...,3,{'context': 'PhishCo runs a number of farms in...,False,train_1263_sub4,[Ponds on low-lying lands would make it harder...
3000,train_1341,Some government economists view their home cou...,Why does the argument's conclusion follow logi...,3,{'context': 'Some government economists view t...,True,train_1341_sub3,[Economics and the physical sciences are diffe...
3001,train_1330,"A mass of ""black water"" containing noxious org...",What is the reasoning for why the argument doe...,3,"{'context': 'A mass of ""black water"" containin...",False,train_1330_sub4,"[The level of vulnerability of the coral, olde..."


In [26]:
# CUT it0, it1 rationale 분석

file_it0 = '/hdd/hjl8708/workspace/ReClor_experiment/result/it0/it0_judgment-1000.jsonl'
file_it1 = '/hdd/hjl8708/workspace/ReClor_experiment/result/it1/it1_judgment-1000.jsonl'
file_it2 = '/hdd/hjl8708/workspace/ReClor_experiment/result/it2/it2_judgment-1000.jsonl'

it0 = pd.read_json(file_it0, lines=True)
it1 = pd.read_json(file_it1, lines=True)
it2 = pd.read_json(file_it2, lines=True)

# ReClor에서의 실험에서 CUT 방법론이 정답 예측/Rationale을 어떻게 피드백하고 향상시키는지 분석한다

In [70]:
# it0, it1, it2의 정답 예측이 모두 같은 경우
it0_correct = it0[it0['label']==it0['prediction']]
it0_incorrect = it0[it0['label']!=it0['prediction']]
it1_correct = it1[it1['label']==it1['prediction']]
it1_incorrect = it1[it1['label']!=it1['prediction']]
it2_correct = it2[it2['label']==it2['prediction']]
it2_incorrect = it2[it2['label']!=it2['prediction']]

# it0에서의 예측과 it1에서의 예측이 다른 경우
it_changed = it0[it0['prediction']!=it1['prediction']]

# it0에서는 틀렸지만 it1 혹은 it2에서는 맞은 경우 --> it0_incorrect에 있지만, it1_correct 혹은 it1_correct에 있음
it_corrected = it0_incorrect[it0_incorrect['id_string'].isin(it1_correct['id_string']) | it0_incorrect['id_string'].isin(it2_correct['id_string'])]

# it0에서는 맞췄지만 it1 혹은 it2에서는 틀린 경우 --> it0_correct에 있지만, it1_incorrect 혹은 it2_incorrect에 있음
it_incorrected = it0_correct[it0_correct['id_string'].isin(it1_incorrect['id_string']) | it0_correct['id_string'].isin(it2_incorrect['id_string'])]

In [72]:
it_corrected

,context,question,answers,label,id_string,input_string,inference,prediction,judged
0,"If the jury did not return a verdict, there wo...",The pattern of reasoning in the argument above...,"[If Ralph had told Manuela about the problem, ...",2,train_985,"Below is an instruction that describes a task,...",The answer is option 2: \nIf Kay's television...,1,The answer is [INCORRECT]. The model incorrect...
4,A high percentage of injuries to the anterior ...,Which of the following most logically complete...,[gymnasts are often given credit for inventing...,1,train_1761,"Below is an instruction that describes a task,...","The answer is option 3: Gymnasts are, in gener...",2,The model's inference is [INCORRECT]. The rati...
5,"Sam: In a recent survey, over 95 percent of pe...",Which one of the following most accurately des...,"[It argues that Sam's conclusion is correct, t...",1,train_3605,"Below is an instruction that describes a task,...",The answer is option 1: Tiya'a response argues...,0,The model's inference is [INCORRECT] because T...
8,It is obvious that one ought to have a will st...,Which one of the following is an assumption on...,[No one wants his or her estate to go to someo...,1,train_2508,"Below is an instruction that describes a task,...",The answer is option 3: One's beloved friend...,2,The model's inference is [INCORRECT]. The rati...
18,Politician: Now that we are finally cleaning u...,Which one of the following most closely parall...,"[In order to like abstract art, you have to un...",1,train_2645,"Below is an instruction that describes a task,...",The answer is option 3: If there had bee a fu...,2,The model's inference is [INCORRECT]. The rati...
...,...,...,...,...,...,...,...,...,...
912,Environmentalist: Snowmobiles in the park nort...,"Which of the following, if true, could best be...",[Not all of the people who go snowmobiling in ...,2,train_2535,"Below is an instruction that describes a task,...","The answer is option 4: Snowmobils, because t...",3,The model's inference is [INCORRECT] because i...
914,The most common bird in Stillwater Marsh is a ...,"Each of the following, if true, helps to recon...",[Although many small marsh birds fly in groups...,1,train_3913,"Below is an instruction that describes a task,...",The answer is option 4: Many marsh birds are...,3,The model's inference is [INCORRECT] because o...
951,"Historian: Flavius, an ancient Roman governor ...",The historian's argumentation is most vulnerab...,[fails to consider the percentage of plays wri...,2,train_2389,"Below is an instruction that describes a task,...",The answer is option 2: Flavious's argument fa...,1,The model's inference is [INCORRECT] because i...
953,"To reduce waste of raw materials, the governme...","Which of the following, if true, most seriousl...",[Many nonfunctioning appliances that are curre...,2,train_4333,"Below is an instruction that describes a task,...",The answer is option 2: The fee provides no i...,1,The model's inference is [INCORRECT] because i...


In [73]:
# it0에서 맞춘 것 3개, 틀린 것 3개 'id_string'을 뽑기 (랜덤)
it0_correct_id = list(it0_correct['id_string'].values)
it0_incorrect_id = list(it0_incorrect['id_string'].values)
it_changed_id = list(it_changed['id_string'].values)
it_corrected_id = list(it_corrected['id_string'].values)
it_incorrected_id = list(it_incorrected['id_string'].values)

import random 
random.seed(42)
random.shuffle(it0_correct_id)
random.shuffle(it0_incorrect_id)
random.shuffle(it_changed_id)
random.shuffle(it_corrected_id)
random.shuffle(it_incorrected_id)

it0_correct_id = it0_correct_id[:3]
it0_incorrect_id = it0_incorrect_id[:3]
it_changed_id = it_changed_id[:3]
it_corrected_id = it_corrected_id[:3]
it_incorrected_id = it_incorrected_id[:3]


In [77]:
# for cid in it0_correct_id[0]:
cid = it_incorrected_id[1]
data = it_incorrected
# context
print("Context:", data[data['id_string']==cid]['context'].values[0])
# question
print("Question:", data[data['id_string']==cid]['question'].values[0])
# label
print("Label: ", data[data['id_string']==cid]['label'].values[0]+1)
print("--------------it0--------------")
# it0 prediction
print('<prediction>:', it0[it0['id_string']==cid]['prediction'].values[0]+1)
# it0 inference
print('<inference>:', it0[it0['id_string']==cid]['inference'].values[0])
# it0 judged
print('<judgment>:', it0[it0['id_string']==cid]['judged'].values[0])
print("--------------it1--------------")
# it1 prediction
print('<prediction>:', it1[it1['id_string']==cid]['prediction'].values[0]+1)
# it1 inference
print('<inference>:', it1[it1['id_string']==cid]['inference'].values[0])
# it1 judged
print('<judgment>:',it1[it1['id_string']==cid]['judged'].values[0])
# it2 prediction
print("--------------it2--------------")
print('<prediction>:', it2[it2['id_string']==cid]['prediction'].values[0]+1)
# it2 inference
print('<inference>:', it2[it2['id_string']==cid]['inference'].values[0])
# it2 judged
print('<judgment>:',it2[it2['id_string']==cid]['judged'].values[0])
print("------------------------------")
    
    
    

Context: Maude is incessantly engaging in diatribes against people who are materialistic. But her hypocrisy is evinced by the sentimental treatment of the watch her grandmother gave her. She certainly is very fond of the watch -- she worries about damaging it; in fact she always sets it carefully in a special box before going to bed.
Question: Which one of the following is an assumption on which the argument depends?
Label:  4
--------------it0--------------
<prediction>: 4
<inference>: The answer is option 4:  Sentimental attachments indicate being materialistic because the argument is based on the assumption that people who are not  materialistic tend not to have sentimental attachements to things. The passage states that Maude is fond of the  watch and worries about  damaging it, which suggests that she has a sentimental attachment to it. However, the passage also states that Mauede engages in diatrabes against people  who are materialists, which implies that she believes that mater

In [30]:
# df가 주어지면 id_string에 해당하는 row를 가져오는 함수
def get_row(df, id_string):
    return df[df['id_string']==id_string]
